In [44]:
import pandas as pd

In [ ]:
#for player stats
# df_2023 = pd.read_csv("../../data/nba/api_data/player_stats/nba_player_stats_2023.csv")
# df_2024 = pd.read_csv("../../data/nba/api_data/player_stats/nba_player_stats_2024.csv")

#for team stats
df_2023 = pd.read_csv("../../data/nba/api_data/player_stats/nba_player_stats_2023.csv")
df_2024 = pd.read_csv("../../data/nba/api_data/player_stats/nba_player_stats_2024.csv")

C:\Users\GauravRadadiya\AppData\Local\Temp\ipykernel_21036\2716845371.py:6: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv("../../data/nba/api_data/player_stats/nba_player_stats_2023.csv")
C:\Users\GauravRadadiya\AppData\Local\Temp\ipykernel_21036\2716845371.py:7: DtypeWarning: Columns (24,345) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2024 = pd.read_csv("../../data/nba/api_data/player_stats/nba_player_stats_2024.csv")


In [49]:
print(df_2023.columns.tolist())
print(df_2024.columns.tolist())


['game_id', 'game_sr_id', 'scheduled', 'duration', 'game_date', 'status', 'attendance', 'track_on_court', 'team_id', 'team_sr_id', 'team_name', 'team_alias', 'team_market', 'points', 'bonus', 'timeouts_remaining', 'record_wins', 'record_losses', 'venue_id', 'venue_name', 'venue_capacity', 'venue_address', 'venue_city', 'venue_state', 'venue_zip', 'venue_country', 'venue_sr_id', 'venue_lat', 'venue_lon', 'player_full_name', 'player_jersey_number', 'player_id', 'player_first_name', 'player_last_name', 'player_position', 'player_primary_position', 'player_played', 'player_active', 'player_on_court', 'player_sr_id', 'player_reference', 'minutes', 'field_goals_made', 'field_goals_att', 'field_goals_pct', 'three_points_made', 'three_points_att', 'three_points_pct', 'two_points_made', 'two_points_att', 'two_points_pct', 'blocked_att', 'free_throws_made', 'free_throws_att', 'free_throws_pct', 'offensive_rebounds', 'defensive_rebounds', 'rebounds', 'assists', 'turnovers', 'steals', 'blocks', 'a

In [47]:
import pandas as pd
import re
from dateutil.parser import parse

# Load CSV (adjust the path for 2023 or 2024) player stats
# df = pd.read_csv("../../data/nba/api_data/player_stats/nba_player_stats_2024.csv")

#team stats
df = pd.read_csv("../../data/nba/api_data/team_stats/nba_team_stats_2024.csv")


# Regex patterns
uuid_pattern = re.compile(r'^[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}$')
year_pattern = re.compile(r'\d{4}')  # look for year to detect proper timestamps

# Helper function to infer SQL type
def infer_sql_type(col, df, sample_size=50):
    col_sample = df[col].dropna().head(sample_size)
    
    if len(col_sample) == 0:
        return "TEXT"  # default if no values exist
    
    for val in col_sample:
        str_val = str(val).strip()
        
        # UUID detection
        if uuid_pattern.match(str_val):
            return "TEXT"
        
        # Boolean detection
        if str_val.lower() in ["true", "false"]:
            return "BOOLEAN"
        
        # Numeric detection (integers and floats)
        try:
            float_val = float(str_val)
            return "NUMERIC"
        except:
            pass
        
        # Timestamp detection (only if string contains a 4-digit year)
        if year_pattern.search(str_val):
            try:
                parse(str_val)
                return "TIMESTAMP"
            except:
                pass
    
    # Default fallback
    return "TEXT"

# Build SQL query while preserving column order
query_lines = []
for col in df.columns:
    col_name = f'"{col}"' if col[0].isdigit() else col  # quote if starts with digit
    sql_type = infer_sql_type(col, df)
    query_lines.append(f"    {col_name} {sql_type}")

query = "CREATE TABLE basketball_nba_team_stats (\n" + ",\n".join(query_lines) + "\n);"

# Output the CREATE TABLE query
print(query)


C:\Users\GauravRadadiya\AppData\Local\Temp\ipykernel_21036\964366618.py:9: DtypeWarning: Columns (418) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../data/nba/api_data/team_stats/nba_team_stats_2024.csv")


CREATE TABLE basketball_nba_team_stats (
    game_id TEXT,
    game_sr_id TEXT,
    scheduled TIMESTAMP,
    duration TEXT,
    game_date TIMESTAMP,
    status TEXT,
    attendance NUMERIC,
    track_on_court BOOLEAN,
    team_id TEXT,
    team_sr_id TEXT,
    team_name TEXT,
    team_alias TEXT,
    team_market TEXT,
    points NUMERIC,
    bonus BOOLEAN,
    timeouts_remaining NUMERIC,
    record_wins NUMERIC,
    record_losses NUMERIC,
    venue_id TEXT,
    venue_name TEXT,
    venue_capacity NUMERIC,
    venue_address TEXT,
    venue_city TEXT,
    venue_state TEXT,
    venue_zip NUMERIC,
    venue_country TEXT,
    venue_sr_id TEXT,
    venue_lat NUMERIC,
    venue_lon NUMERIC,
    scoring_0_points NUMERIC,
    scoring_0_type TEXT,
    scoring_0_number NUMERIC,
    scoring_0_sequence NUMERIC,
    scoring_1_points NUMERIC,
    scoring_1_type TEXT,
    scoring_1_number NUMERIC,
    scoring_1_sequence NUMERIC,
    scoring_2_points NUMERIC,
    scoring_2_type TEXT,
    scoring_2_numb